In [1]:
# Load libraries and initialize values


# seed value for random number generators to obtain reproducible results
RANDOM_SEED = 1


# although we standardize X and y variables on input,
# we will fit the intercept term in the models
# Expect fitted values to be close to zero
SET_FIT_INTERCEPT = True

# import base packages into the namespace for this program
import numpy as np
import pandas as pd

# modeling routines from Scikit Learn packages

# Assignment 4: Be sure to add decision tree and random forest packages

from sklearn.datasets import fetch_mldata, fetch_openml
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.decomposition import PCA

from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import precision_score, recall_score, f1_score, fbeta_score
from sklearn.metrics import matthews_corrcoef, brier_score_loss
# from sklearn.metrics import roc_curve, roc_auc_score  # Not relevant to a multiclass classifier
# from sklearn.metrics import precision_recall_curve, average_precision_score
from sklearn.model_selection import train_test_split, KFold
from sklearn.utils.multiclass import unique_labels

from sklearn.preprocessing import MinMaxScaler, Normalizer, StandardScaler, RobustScaler

from sklearn.ensemble import BaggingRegressor, RandomForestRegressor, ExtraTreesRegressor
from sklearn.ensemble import AdaBoostRegressor, GradientBoostingRegressor, VotingRegressor
import sklearn.linear_model
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet, LassoLarsIC
from sklearn.linear_model import RidgeCV, LassoCV, ElasticNetCV
from sklearn.metrics import f1_score, mean_squared_error, r2_score
from sklearn.metrics.scorer import make_scorer
from sklearn.model_selection import cross_val_score, train_test_split, GridSearchCV

from sklearn.model_selection import cross_val_predict

from sklearn.pipeline import Pipeline
from sklearn.tree import DecisionTreeRegressor, export_graphviz
import time
from functools import wraps

# Let's comment out the math library. Stick with the numpy version of the function.

# from math import sqrt  # log function; sqrt for root mean-squared error calculation

# np.sqrt works on vectors and thus plays well with matplotlib
# Attempts to transform more than single numbers with math.sqrt often crash plt plots
# Speaking of plt… Here are libraries for visualizations

import matplotlib as mpl
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import graphviz
import pydot

# Sanity check: Echo output so that we know that this critical first chunk has run successfully
print("All libraries imported. RANDOM_SEED = {:d}".format(RANDOM_SEED))

ModuleNotFoundError: No module named 'graphviz'

We'll start with a small housekeeping task: Creating a decorator function for tracking time. This code, alas, cannot distinguish between calls to the same function by different parts of the program. However, having a count of time spent on certain repeated tasks will help us diagnose which parts of the random forest and PCA-dimension reduction tasks consume the most time.

In [ ]:
# Time decorator code adapted from:
#   https://stackoverflow.com/questions/3620943/measuring-elapsed-time-with-the-time-module
# Key changes:
#   time.monotonic() instead of time.time()
#   print_prof_data() function revamped to show function / calls / total / average / max in tabular format

PROF_DATA = {}

def profile(fn):
    @wraps(fn)
    def with_profiling(*args, **kwargs):
        start_time = time.monotonic()
        
        ret = fn(*args, **kwargs)
        
        elapsed_time = time.monotonic() - start_time

        if fn.__name__ not in PROF_DATA:
            PROF_DATA[fn.__name__] = [0, []]
        PROF_DATA[fn.__name__][0] += 1
        PROF_DATA[fn.__name__][1].append(elapsed_time)

        return ret

    return with_profiling

def print_prof_data():
    print("\n{:>25}\t{:>8}\t{:>12}\t{:>12}\t{:>12}\t{:>12}".format("Function", "Calls", "Total", "Average", "Std dev", "Maximum"))
    print(108 * "_" + "\n")
    total_calls = 0
    grand_time = 0
    for fname, data in PROF_DATA.items():
        total_calls += data[0]
        total_time = sum(data[1])
        grand_time += total_time
        avg_time = np.mean(data[1])
        std_time = np.std(data[1])
        max_time = max(data[1])
        print("{:>25s}\t{:8d}\t{:12.6f}\t{:12.6f}\t{:12.6f}\t{:12.6f}".format(fname, data[0], total_time, avg_time, std_time, max_time))
    print(108 * "_")
    print("\n{:>25}\t{:8d}\t{:>12.6f}\t{:12.6f}".format("ALL FUNCTIONS COMBINED", total_calls, grand_time, grand_time / total_calls))

def clear_prof_data():
    global PROF_DATA
    PROF_DATA = {}

In [ ]:
# Test the time decorator function — Set up

@profile
def factorial(n):
    """Assumes n is an integer > 0
        Returns n!"""
    if n == 0:
        return 1
    elif n == 1:
        return n
    else:
        return n * factorial(n - 1)

@profile
def permutations(n, r):
    """ Assumes n, k are integers > 0
        Returns nPr """
    nPr = factorial(n)/factorial(n - r)
    nPr = int(round(nPr))
    return nPr

@profile
def combinations(n, r):
    """ Assumes n, k are integers > 0
        Returns nCr """
    nCr = permutations(n, r)/factorial(r)
    nCr = int(round(nCr))
    return nCr

# Test the time decorator function — Write a tiny Fibonnaci triangle

for i in range(0, 13):
    for j in range(i + 1):
        print(combinations(i, j), "\t", end = "")
        # print("C({:d}, {:d}) = {:d}\t".format(i, j, combinations(i, j)), end = "")
    print("\n")

print_prof_data()

print("\nNow clearing the timekeeping cache so that we keep accurate track of machine-learning tasks …")
clear_prof_data()

In [ ]:
# Let's begin our machine-learning tasks in earnest!
# Begin by loading the full set of 784 explanatory variables in the MNIST data set

# This appears to be deprecated code, but it shows up in MSDS program and Géron code
# mnist = fetch_openml('MNIST original')

mnist = fetch_openml('mnist_784', version = 1)
# mnist  # This echoes the MNIST dataset
mnist.keys()

In [ ]:
X, y = mnist["data"], mnist["target"]

# Recast y as integers
X = X.astype(np.uint8)
y = y.astype(np.uint8)

print("The shape of mnist X (features):\t", X.shape)
print("The shape of mnist y (target):  \t", y.shape)

In [ ]:
# Display the first entry in mnist

pick_a_digit = 4

print("This should be a", y[pick_a_digit])
some_digit = X[pick_a_digit]
some_digit_image = some_digit.reshape(28, 28)
plt.imshow(some_digit_image, cmap = mpl.cm.binary, interpolation = "nearest")
plt.axis("off")
plt.savefig("some_digit_plot.png")
plt.savefig("some_digit_plot.pdf")
plt.show()

In [ ]:
# Demonstration code for showing off a set of mnist images

def plot_digits(instances, images_per_row=10, **options):
    size = 28
    images_per_row = min(len(instances), images_per_row)
    images = [instance.reshape(size,size) for instance in instances]
    n_rows = (len(instances) - 1) // images_per_row + 1
    row_images = []
    n_empty = n_rows * images_per_row - len(instances)
    images.append(np.zeros((size, size * n_empty)))
    for row in range(n_rows):
        rimages = images[row * images_per_row : (row + 1) * images_per_row]
        row_images.append(np.concatenate(rimages, axis=1))
    image = np.concatenate(row_images, axis=0)
    plt.imshow(image, cmap = mpl.cm.binary, **options)
    plt.axis("off")

plt.figure(figsize=(9,9))
example_images = np.r_[X[:12000:600], X[13000:30600:600], X[30600:60000:590]]
plot_digits(example_images, images_per_row=10)
plt.savefig("more_digits_plot.png")
plt.savefig("more_digits_plot.pdf")
plt.show()

Having loaded MNIST, we will now split that dataset into training and test sets. We will also define our primary scorer as one based on the F1 score. Finally, we will frontload numerous scoring, reporting, and plotting functions so that their designation does _not_ count against the time recorded for each of the machine-learning routines.

In [ ]:
# Split mnist into training and test sets

X_train, X_test, y_train, y_test = X[:60000], X[60000:], y[:60000], y[60000:]

# This is an exceptionally important line! We need an F1 scorer in place of "accuracy"

f1_scorer = make_scorer(f1_score, average = "micro", greater_is_better = True)

In [ ]:
crossval_frame = pd.DataFrame(np.zeros((5, 5), dtype = float),
                              columns = ["random_forest", "flawed_PCA", "correct_PCA", "PCA_pipe", "sgd"],
                              index = ["f1_score", "standard_deviation", "score_0", "score_1", "score_2"])

In [ ]:
# A defined function for Kfold cross-validation: Use 3 folds in light of the heightened computational expense

np.set_printoptions(precision = 6)
CV_FOLDS = 3

@profile
def crossval_scoring(model, label, column_name, features = X_train):
    print("{:>30}\t{:>10}\t{:>8}\t{:>18}".format("Multiclass classifier", "F1 score", "Std Dev", "Scores"))
    print(92 * "_")
    score_column = np.zeros(5, dtype = float)
    scores = cross_val_score(model, features, y_train, scoring = "f1_micro", cv = CV_FOLDS)
    score_column[0] = scores.mean()
    score_column[1] = scores.std()
    score_column[2] = scores[0]
    score_column[3] = scores[1]
    score_column[4] = scores[2]
    crossval_frame[column_name] = score_column
    print("{:>30}\t{:10.6f}\t{:8.6f}\t{}".format(label, scores.mean(), scores.std(), scores))

In [ ]:
# Some ways to display confusion matrixes graphically
# Leave the routines adapted from SciKit-Learn documentation and MSDS sample code for future use
# The "purple_haze" routine combines the best of both worlds. We'll stick with that version. See bottom.

def plot_confusion_matrix(y_true, y_pred, classes, standardize = False,
                          title = None, file_name = "default", cmap = plt.cm.Blues):
    
    # This code enables the display of the confusion matrix in graphic format
    # Code adapted from SciKit-Learn documentation:
    # https://scikit-learn.org/stable/auto_examples/model_selection/plot_confusion_matrix.html
    
    if not title:
        if standardize:
            title = "Standardized confusion matrix"
        else:
            title = "Unstandardized confusion matrix"
    title += "\n"
    cm = confusion_matrix(y_true, y_pred)
    classes = classes[unique_labels(y_true, y_pred)]
    if standardize:
        cm = cm.astype("float") / cm.sum(axis = 1)[:, np.newaxis]
        
    # Sklearn documentation offers to print the confusion matrix. Pass for now.
    # print(title)
    # print(cm)
    
    fig, ax = plt.subplots(figsize = (9, 9))
    im = ax.imshow(cm, interpolation = "nearest", cmap = cmap)
    ax.figure.colorbar(im, ax = ax)
    ax.set(xticks = np.arange(cm.shape[1]), yticks = np.arange(cm.shape[0]),
           xticklabels = classes, yticklabels = classes, title = title,
           ylabel = "True label\n", xlabel = "\nPredicted label")
    plt.setp(ax.get_xticklabels(), rotation = 0, ha = "right",
             rotation_mode = "anchor")
    fmt = ".3f" if standardize else "d"
    thresh = cm.max() / 2 if not standardize else 0.5
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            ax.text(j, i, format(cm[i, j], fmt), ha = "center", va = "center",
                    color = "white" if cm[i, j] > thresh else "black")
    fig.tight_layout()
    plt.savefig(file_name + "_confmat.png")
    plt.savefig(file_name + "_confmat.pdf")
    return ax

def colorful_confusion_matrix(matrix, title = "Colorful confusion matrix", file_name = "default"):
    fig = plt.figure(figsize=(9, 9))
    ax = fig.add_subplot(111)
    ax.set(title = title + "\n", ylabel = "True label\n", xlabel = "\nPredicted label")
    cax = ax.matshow(matrix)
    fig.colorbar(cax)
    plt.savefig(file_name + "_colorCM.png")
    plt.savefig(file_name + "_colorCM.pdf")

def standardized_confusion_matrix(matrix):
    row_sums = matrix.sum(axis = 1, keepdims = True)
    standardized_matrix = matrix / row_sums
    np.fill_diagonal(standardized_matrix, 0)
    return standardized_matrix

@profile
def purple_haze(y_true, y_pred, classes, standardize = False, title = None, file_name = "default",
                cmap = plt.cm.plasma):
    
    # This code enables the display of the confusion matrix in graphic format
    # Code adapted from SciKit-Learn documentation:
    # https://scikit-learn.org/stable/auto_examples/model_selection/plot_confusion_matrix.html
    
    if not title:
        if standardize:
            title = "Standardized confusion matrix"
        else:
            title = "Unstandardized confusion matrix"
    title += "\n"
    cm = confusion_matrix(y_true, y_pred)
    classes = classes[unique_labels(y_true, y_pred)]
    if standardize:
        np.fill_diagonal(cm, 0)  # Add this line; see what happens
        cm = cm.astype("float") / cm.sum(axis = 1)[:, np.newaxis]
        
    # Sklearn documentation offers to print the confusion matrix. Pass for now.
    # print(title)
    # print(cm)
    
    fig, ax = plt.subplots(figsize = (9, 9))
    im = ax.imshow(cm, interpolation = "nearest", cmap = cmap)
    ax.figure.colorbar(im, ax = ax)
    ax.set(xticks = np.arange(cm.shape[1]), yticks = np.arange(cm.shape[0]),
           xticklabels = classes, yticklabels = classes, title = title,
           ylabel = "True label\n", xlabel = "\nPredicted label")
    plt.setp(ax.get_xticklabels(), rotation = 0, ha = "right",
             rotation_mode = "anchor")
    fmt = ".3f" if standardize else "d"
    thresh = cm.max() / 2 # if not standardize else 0.4
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            ax.text(j, i, format(cm[i, j], fmt), ha = "center", va = "center",
                    color = "black" if cm[i, j] > thresh else "white")
    fig.tight_layout()
    plt.savefig(file_name + "_confmat.png")
    plt.savefig(file_name + "_confmat.pdf")
    return ax

In [ ]:
# Create a scorecard_frame in anticipation of scoring functions
# We'll be glad in the final cells that we took the trouble to do this now

scorecard_frame = pd.DataFrame(np.zeros((9, 5), dtype = float),
                              columns = ["random_forest", "flawed_PCA", "correct_PCA", "PCA_pipe", "sgd"],
                              index = ["training_score", "test_score", "precision", "recall", "f1_score",
                                       "f2_score", "specificity", "bookmaker_informedness",
                                       "matthews_correlation"])

In [ ]:
# Scoring functions

def specificity_score(y_true, y_pred):
    ss = confusion_matrix(y_true, y_pred)[0, 0] / sum(confusion_matrix(y_true, y_pred)[0])
    return ss

def bookmaker_informedness(y_true, y_pred):
    bm = recall_score(y_true, y_pred, average = None) + specificity_score(y_true, y_pred) - 1
    return bm

@profile
def scorecard(model, train, test, column_name):
    fitted_model = model.fit(train, y_train)
    model_name = str(model).partition("(")[0] + " | " + column_name
    model_predict = model.predict(test)
    score_column = np.zeros(9, dtype = float)
    print("Classification scorecard for " + model_name + ":\n")
    
    train_score = model.score(train, y_train)
    score_column[0] = train_score
    print("{:>25}\t{:.6f}".format("Training set score", train_score))
    
    test_score = model.score(test, y_test)
    score_column[1] = test_score
    print("{:>25}\t{:.6f}".format("Test set score", test_score))
    
    precision = precision_score(y_test, model_predict, average = None).mean()
    score_column[2] = precision
    print("{:>25}\t{:.6f}".format("Precision", precision))
    
    recall = recall_score(y_test, model_predict, average = None).mean()
    score_column[3] = recall
    print("{:>25}\t{:.6f}".format("Recall", recall))
    
    f1 = f1_score(y_test, model_predict, average = None).mean()
    score_column[4] = f1
    print("{:>25}\t{:.6f}".format("F1 score", f1))
    
    f2 = fbeta_score(y_test, model_predict, beta = 2, average = None).mean()
    score_column[5] = f2
    print("{:>25}\t{:.6f}".format("F2 score", f2))
    
    specificity = specificity_score(y_test, model_predict)
    score_column[6] = specificity
    print("{:>25}\t{:.6f}".format("Specificity", specificity))
    
    bmi = bookmaker_informedness(y_test, model_predict).mean()
    score_column[7] = bmi
    print("{:>25}\t{:.6f}".format("Bookmaker informedness", bmi))
    
    matt_corr = matthews_corrcoef(y_test, model_predict)
    score_column[8] = matt_corr
    print("{:>25}\t{:.6f}".format("Matthews correlation", matt_corr))
    
    scorecard_frame[column_name] = score_column

# The following code was the original code for the scorecard
# Keep it around — It worked (very) well. Its only "flaw" was not recording its output for a dataframe

def scorecard_backup(model, train, test):
    fitted_model = model.fit(train, y_train)
    model_name = str(model).partition("(")[0]
    model_predict = model.predict(test)
    print("Classification scorecard for " + model_name + ":\n")
    print("{:>25}\t{:.6f}".format("Training set score", model.score(train, y_train)))
    print("{:>25}\t{:.6f}".format("Test set score", model.score(test, y_test)))
    print("{:>25}\t{:.6f}".format("Precision", precision_score(y_test, model_predict, average = None).mean()))
    print("{:>25}\t{:.6f}".format("Recall", recall_score(y_test, model_predict, average = None).mean()))
    print("{:>25}\t{:.6f}".format("F1 score", f1_score(y_test, model_predict, average = None).mean()))
    print("{:>25}\t{:.6f}".format("F2 score", fbeta_score(y_test, model_predict, beta = 2, average = None).mean()))
    print("{:>25}\t{:.6f}".format("Specificity", specificity_score(y_test, model_predict)))
    print("{:>25}\t{:.6f}".format("Bookmaker informedness", bookmaker_informedness(y_test, model_predict).mean()))
    print("{:>25}\t{:.6f}".format("Matthews correlation", matthews_corrcoef(y_test, model_predict)))


In [ ]:
# Start the clock on the original random forest process

RF_start_time = time.monotonic()

In [ ]:
# Let's load and test a Random Forest Classifier!

# Use GridSearchCV() to evaluate random forest classifers

@profile
def GridSearchForest(features, target):
    feature_count = len(features[0])
    low_depth = int(round(np.log2(feature_count)))
    mid_depth = int(round(np.sqrt(2) * np.log2(feature_count)))
    high_depth = int(round(2 * np.log2(feature_count)))
    low_features = int(round(np.sqrt(feature_count)))
    mid_features = int(round(np.sqrt(np.e) * np.sqrt(feature_count)))
    high_features = int(round(np.e * np.sqrt(feature_count)))
    param_grid = {"max_depth": [low_depth, mid_depth, high_depth],
                  "max_features": [low_features, mid_features, high_features]}
    grid_search_forest = GridSearchCV(RandomForestClassifier(random_state = RANDOM_SEED, n_estimators = 20),
                                      param_grid, cv = 3, return_train_score = True, scoring = f1_scorer, iid = False)
    grid_search_forest.fit(features, target)
    forest_depth = grid_search_forest.best_params_["max_depth"]
    forest_features = grid_search_forest.best_params_["max_features"]
    forest_score = grid_search_forest.best_score_
    forest_grid_cv_results = grid_search_forest.cv_results_
    return forest_depth, forest_features, forest_score, forest_grid_cv_results

forest_depth, forest_features, forest_score, forest_grid_cv_results = GridSearchForest(X_train, y_train)

In [ ]:
print("Maximum depth:   \t{:d}".format(forest_depth))
print("Maximum features:\t{:d}".format(forest_features))
print("Best cv score:   \t{:.6f}".format(forest_score))

In [ ]:
# Capture the GridSearch results in a dataframe and display them

forest_grid_search_results = pd.DataFrame(forest_grid_cv_results)

forest_grid_search_results.rename(columns = {"param_max_depth": "max_depth",
                                             "param_max_features": "max_features"}, inplace = True)
forest_grid_search_results["total_time"] = forest_grid_search_results["mean_fit_time"] + forest_grid_search_results["mean_score_time"]
forest_grid_search_results["standardized_time"] = forest_grid_search_results.total_time.map(lambda x: x / sum(forest_grid_search_results.total_time))
forest_grid_search_results.loc[:, ["rank_test_score", "max_depth", "max_features", "mean_test_score", "mean_train_score", "mean_fit_time", "mean_score_time", "total_time", "standardized_time"]].sort_values(by = "rank_test_score")

In [ ]:
# Plot the optimization of the basic Random Forest Classifier in 3D

fig = plt.figure(figsize = (10, 10))
ax = fig.gca(projection='3d')

plot3d_title = "Random Forest Classifier cross-validation via GridSearch\n"
plot3d_title += "F1 score ≈ " + str(round(forest_score, 6))
plot3d_title += " at max_depth = " + str(forest_depth) + " and max_features = " + str(forest_features)

ax.set_title(plot3d_title)
ax.set_xlabel("max_depth", color = "r")
ax.set_ylabel("max_features", color = "b")
ax.set_zlabel("F1 score", color = "g")

ax.plot(forest_grid_search_results["max_depth"], forest_grid_search_results["max_features"],
           forest_grid_search_results["mean_test_score"], color = "blue", label = "Hyperparameter testing path")

ax.scatter(forest_depth, forest_features, forest_score, s = 256, marker = "o",
           color = "#ff0000cc", label = "Optimal depth, features")

depth_space =    np.linspace(forest_grid_search_results["max_depth"].min(),
                             forest_grid_search_results["max_depth"].max() + 1, 120)
features_space = np.linspace(forest_grid_search_results["max_features"].min(),
                             forest_grid_search_results["max_features"].max() + 1, 120)

plot_x, plot_y = np.meshgrid(depth_space, features_space)
score_plane = 0 * plot_x + 0 * plot_y + forest_score

ax.plot_wireframe(plot_x, plot_y, score_plane, rcount = 6, ccount = 6, color = "#00884466")

ax.legend(loc = "lower right")
ax.view_init(12, 45)
plt.savefig("forest_hyperparameters.png")
plt.savefig("forest_hyperparameters.pdf")

In [ ]:
# Plot the optimization of the basic Random Forest Classifier in 3D — Tracking time

fig = plt.figure(figsize = (10, 10))
ax = fig.gca(projection='3d')

ax.set_xlabel("\nProcessing time (standardized)", color = "r")
ax.set_ylabel("\nTrain minus test ~ Overfitting", color = "b")
ax.set_zlabel("\nF1 score", color = "g")

ax.plot(forest_grid_search_results["standardized_time"],
        forest_grid_search_results["mean_train_score"] - forest_grid_search_results["mean_test_score"],
        forest_grid_search_results["mean_test_score"], color = "blue", label = "Hyperparameter testing path")

winner_index = forest_grid_search_results["mean_test_score"].idxmax()

x_locate = forest_grid_search_results.standardized_time[winner_index]
y_locate = forest_grid_search_results.mean_train_score[winner_index] - forest_grid_search_results.mean_test_score[winner_index]

ax.scatter(x_locate, y_locate, forest_score, s = 256, marker = "o", color = "#ff0000cc",
           label = "Optimal depth, features")

depth_space =    np.linspace(forest_grid_search_results["standardized_time"].min(),
                             forest_grid_search_results["standardized_time"].max(), 120)
features_space = np.linspace(min(forest_grid_search_results["mean_train_score"] - forest_grid_search_results["mean_test_score"]),
                             max(forest_grid_search_results["mean_train_score"] - forest_grid_search_results["mean_test_score"]), 120)

plot3d_title = "Random Forest Classifier cross-validation | "
plot3d_title += "F1 score ≈ " + str(round(forest_score, 6))
plot3d_title +=  "\nStandardized time ≈ " + str(round(x_locate, 6))
plot3d_title += " | Train minus test ≈ " + str(round(y_locate, 6))
ax.set_title(plot3d_title)

plot_x, plot_y = np.meshgrid(depth_space, features_space)
score_plane = 0 * plot_x + 0 * plot_y + forest_score

ax.plot_wireframe(plot_x, plot_y, score_plane, rcount = 8, ccount = 7, color = "#00884466")

ax.legend(loc = "lower right")
ax.view_init(12, 45)
plt.savefig("forest_hyperparameters_time.png")
plt.savefig("forest_hyperparameters_time.pdf")

In [ ]:
# Implement a random forest classifier based on hyperparameters reported by GridSearch

@profile
def forest_fit(features, target):
    forest = RandomForestClassifier(random_state = RANDOM_SEED, max_depth = forest_depth,
                                    max_features = forest_features, n_estimators = 20)
    fit = forest.fit(features, target)
    return fit

forest = forest_fit(X_train, y_train)

# Forgo the printing of these scores — They take time, and the information appears in the scorecard
# print("Training set score:\t{:.6f}".format(forest.score(X_train, y_train)))
# print("Test set score:\t\t{:6f}".format(forest.score(X_test, y_test)))

In [ ]:
crossval_scoring(forest, "Random forest classifier", "random_forest", X_train)

In [ ]:
scorecard(forest, X_train, X_test, "random_forest")

In [ ]:
# Finish with a flourish!
# Plot confusion matrixes — ordinary and standardized — for the random forest classifier
# "Purple Haze" is an allusion to a Jimi Hendrix song. Haze = confusion. Purple = plasma or viridis colormap

purple_haze(y_test, forest.predict(X_test), classes = forest.classes_,
            title = "Random forest classifier", file_name = "forest")

# Use this cell to retain function calls using alternative methods of expressing the confusion matrix

# plot_confusion_matrix(y_test, forest.predict(X_test), classes = forest.classes_,
#                       title = "Random forest classifier", file_name = "forest")

# plot_confusion_matrix(y_test, forest.predict(X_test), classes = forest.classes_, standardize = True,
#                       title = "Random forest classifier — Standardized", file_name = "forest_standardized")

# Colorful confusion matrix, via MSDS sample code
# This image requires you to define the matrix, and then call the function

# RF_cm = confusion_matrix(y_test, forest.predict(X_test))
# colorful_confusion_matrix(RF_cm, "Random forest", "forest")

# RF_standard_cm = standardized_confusion_matrix(RF_cm)
# colorful_confusion_matrix(RF_standard_cm, "Random forest — Standardized", "forest_standard")

In [ ]:
purple_haze(y_test, forest.predict(X_test), classes = forest.classes_, standardize = True,
            title = "Random forest classifier — Standardized", file_name = "forest_standardized")

In [ ]:
# Stop the random forest clock

RF_elapsed_time = time.monotonic() - RF_start_time

In [ ]:
# Define functions unique to the PCA processes
# These functions will be "wrapped" for timing purposes …
# But the definition of functions themselves won't count against either process

@profile
def PCA_report(variance_ratio, subtitle = ""):
    plot_title = "Reducing MNIST to " + str(len(variance_ratio)) + " principal components,\n"
    plot_title += "which account for " + str(round(sum(variance_ratio) * 100, 2)) + "% of the variance"
    plt.grid()
    plt.title(plot_title)
    plt.plot(np.cumsum(variance_ratio), color = "blue", linestyle = "-", linewidth = 1,
             label = "Cumulative sum of explained variance")
    plt.axhline(0.95, color = "red", linestyle = "--", linewidth = 1, label = "95% cumulative variance")
    plt.ylabel("Explained variance")
    plt.xlabel("Principal components")
    plt.legend(loc = "best")
    plt.savefig(subtitle + "PCA_cumsum.png")
    plt.savefig(subtitle + "PCA_cumsum.pdf")

In [ ]:
# Start the clock on the flawed PCA process

reduced_start_time = time.monotonic()

In [ ]:
@profile
def PCA_reduce(features):
    pca = PCA(n_components = 0.95)
    reduced = pca.fit_transform(features)
    variance_ratio = pca.explained_variance_ratio_
    return reduced, variance_ratio

X_reduced, X_variance_ratio = PCA_reduce(X)

In [ ]:
PCA_report(X_variance_ratio)

In [ ]:
X_reduced_train = X_reduced[:60000]
X_reduced_test = X_reduced[60000:]

In [ ]:
# reduced_forest_features, reduced_forest_score = GridSearchForest(X_reduced_train, y_train)
reduced_forest_depth, reduced_forest_features, reduced_forest_score, reduced_forest_grid_cv_results = GridSearchForest(X_reduced_train, y_train)

In [ ]:
# print("Maximum features:\t{:.6f}".format(reduced_forest_features))
# print("Best cv score:   \t{:.6f}".format(reduced_forest_score))

print("Maximum depth:   \t{:d}".format(reduced_forest_depth))
print("Maximum features:\t{:d}".format(reduced_forest_features))
# print("Number of trees: \t{:d}".format(reduced_forest_estimators))
print("Best cv score:   \t{:.6f}".format(reduced_forest_score))

In [ ]:
# Capture the GridSearch results in a dataframe and display them

reduced_forest_grid_search_results = pd.DataFrame(reduced_forest_grid_cv_results)

reduced_forest_grid_search_results.rename(columns = {"param_max_depth": "max_depth",
                                             "param_max_features": "max_features"}, inplace = True)
reduced_forest_grid_search_results["total_time"] = reduced_forest_grid_search_results["mean_fit_time"] + reduced_forest_grid_search_results["mean_score_time"]
reduced_forest_grid_search_results["standardized_time"] = reduced_forest_grid_search_results.total_time.map(lambda x: x / sum(reduced_forest_grid_search_results.total_time))
reduced_forest_grid_search_results.loc[:, ["rank_test_score", "max_depth", "max_features", "mean_test_score", "mean_train_score", "mean_fit_time", "mean_score_time", "total_time", "standardized_time"]].sort_values(by = "rank_test_score")

In [ ]:
# Plot the optimization of the flawed PCA transformation in 3D

fig = plt.figure(figsize = (10, 10))
ax = fig.gca(projection='3d')

plot3d_title = "(Flawed) PCA-transformed random forest cross-validation via GridSearch\n"
plot3d_title += "F1 score ≈ " + str(round(reduced_forest_score, 6))
plot3d_title += " at max_depth = " + str(reduced_forest_depth) + " and max_features = " + str(reduced_forest_features)

ax.set_title(plot3d_title)
ax.set_xlabel("max_depth", color = "r")
ax.set_ylabel("max_features", color = "b")
ax.set_zlabel("F1 score", color = "g")

ax.plot(reduced_forest_grid_search_results["max_depth"], reduced_forest_grid_search_results["max_features"],
           reduced_forest_grid_search_results["mean_test_score"], color = "blue", label = "Hyperparameter testing path")

ax.scatter(reduced_forest_depth, reduced_forest_features, reduced_forest_score, s = 256, marker = "o",
           color = "#ff0000cc", label = "Optimal depth, features")

depth_space =    np.linspace(reduced_forest_grid_search_results["max_depth"].min(),
                             reduced_forest_grid_search_results["max_depth"].max() + 1, 120)
features_space = np.linspace(reduced_forest_grid_search_results["max_features"].min(),
                             reduced_forest_grid_search_results["max_features"].max() + 1, 120)

plot_x, plot_y = np.meshgrid(depth_space, features_space)
score_plane = 0 * plot_x + 0 * plot_y + reduced_forest_score

ax.plot_wireframe(plot_x, plot_y, score_plane, rcount = 6, ccount = 6, color = "#00884466")

ax.legend(loc = "lower right")
ax.view_init(12, 45)
plt.savefig("reduced_hyperparameters.png")
plt.savefig("reduced_hyperparameters.pdf")

In [ ]:
# Plot the optimization of the flawed PCA-reduced Random Forest Classifier in 3D — Tracking time

fig = plt.figure(figsize = (10, 10))
ax = fig.gca(projection='3d')

ax.set_xlabel("\nProcessing time (standardized)", color = "r")
ax.set_ylabel("\nTrain minus test ~ Overfitting", color = "b")
ax.set_zlabel("\nF1 score", color = "g")

ax.plot(reduced_forest_grid_search_results["standardized_time"],
        reduced_forest_grid_search_results["mean_train_score"] - reduced_forest_grid_search_results["mean_test_score"],
        reduced_forest_grid_search_results["mean_test_score"], color = "blue", label = "Hyperparameter testing path")

winner_index = reduced_forest_grid_search_results["mean_test_score"].idxmax()

x_locate = reduced_forest_grid_search_results.standardized_time[winner_index]
y_locate = reduced_forest_grid_search_results.mean_train_score[winner_index] - reduced_forest_grid_search_results.mean_test_score[winner_index]

ax.scatter(x_locate, y_locate, reduced_forest_score, s = 256, marker = "o", color = "#ff0000cc",
           label = "Optimal depth, features")

depth_space =    np.linspace(reduced_forest_grid_search_results["standardized_time"].min(),
                             reduced_forest_grid_search_results["standardized_time"].max(), 120)
features_space = np.linspace(min(reduced_forest_grid_search_results["mean_train_score"] - reduced_forest_grid_search_results["mean_test_score"]),
                             max(reduced_forest_grid_search_results["mean_train_score"] - reduced_forest_grid_search_results["mean_test_score"]), 120)

plot3d_title = "(Improperly) reduced random forest cross-validation | "
plot3d_title += "F1 score ≈ " + str(round(reduced_forest_score, 6))
plot3d_title +=  "\nStandardized time ≈ " + str(round(x_locate, 6))
plot3d_title += " | Train minus test ≈ " + str(round(y_locate, 6))
ax.set_title(plot3d_title)

plot_x, plot_y = np.meshgrid(depth_space, features_space)
score_plane = 0 * plot_x + 0 * plot_y + reduced_forest_score

ax.plot_wireframe(plot_x, plot_y, score_plane, rcount = 8, ccount = 7, color = "#00884466")

ax.legend(loc = "lower right")
ax.view_init(12, 45)
plt.savefig("reduced_hyperparameters_time.png")
plt.savefig("reduced_hyperparameters_time.pdf")

In [ ]:
# Implement a random forest classifier based on hyperparameters reported by GridSearch

@profile
def reduced_forest_fit(features, target, mdepth, mfeatures):
    forest = RandomForestClassifier(random_state = RANDOM_SEED, max_depth = mdepth, max_features = mfeatures,
                                    n_estimators = 20)
    fit = forest.fit(features, target)
    return fit

reduced_forest = reduced_forest_fit(X_reduced_train, y_train, reduced_forest_depth, reduced_forest_features)

# Forgo these lines to save time — The information will appear in the scorecard
# print("Training set score:\t{:.6f}".format(reduced_forest.score(X_reduced_train, y_train)))
# print("Test set score:\t\t{:6f}".format(reduced_forest.score(X_reduced_test, y_test)))

In [ ]:
crossval_scoring(reduced_forest, "(Flawed) PCA transformation", "flawed_PCA", X_reduced_train)

In [ ]:
scorecard(reduced_forest, X_reduced_train, X_reduced_test, "flawed_PCA")

In [ ]:
purple_haze(y_test, reduced_forest.predict(X_reduced_test), classes = reduced_forest.classes_,
            title = "Reduced random forest classifier", file_name = "reduced_forest")

In [ ]:
purple_haze(y_test, reduced_forest.predict(X_reduced_test), classes = reduced_forest.classes_, standardize = True,
            title = "Reduced random forest classifier — Standardized", file_name = "reduced_forest_normal")

In [ ]:
# Stop the clock on the flawed PCA transformation

reduced_elapsed_time = time.monotonic() - reduced_start_time

In [ ]:
# Start the clock on the flawed PCA process

PCA_start_time = time.monotonic()

In [ ]:
# Beyond the "design flaw" — Let's redo PCA reduction, the right way
# We have to limit PCA transformation to the test set
# Then use linear algebra to recenter the test set with the eigenvectors of the PCA-transformed train set
# Details: https://stats.stackexchange.com/questions/55718/pca-and-the-train-test-split

@profile
def PCA_train_test(train, test):
    pca = PCA(n_components = 0.95)
    PCA_train = pca.fit_transform(train)
    PCA_test = pca.transform(test)  # Absolutely vital that the test set be transformed, but *not* fit here
    variance_ratio = pca.explained_variance_ratio_
    return PCA_train, PCA_test, variance_ratio

X_PCA_train, X_PCA_test, X_PCA_variance_ratio = PCA_train_test(X_train, X_test)

print("Proper PCA-reduction of MNIST completed!")

In [ ]:
PCA_report(X_PCA_variance_ratio, subtitle = "correct_")

In [ ]:
# PCA_forest_features, PCA_forest_score = GridSearchForest(X_PCA_train, y_train)

PCA_forest_depth, PCA_forest_features, PCA_forest_score, PCA_forest_grid_cv_results = GridSearchForest(X_PCA_train, y_train)

In [ ]:
# print("Maximum features:\t{:d}".format(PCA_forest_features))
# print("Best cv score:   \t{:.6f}".format(PCA_forest_score))

print("Maximum depth:   \t{:d}".format(PCA_forest_depth))
print("Maximum features:\t{:d}".format(PCA_forest_features))
print("Best cv score:   \t{:.6f}".format(PCA_forest_score))

In [ ]:
# Capture the GridSearch results in a dataframe and display them

PCA_forest_grid_search_results = pd.DataFrame(PCA_forest_grid_cv_results)

PCA_forest_grid_search_results.rename(columns = {"param_max_depth": "max_depth",
                                             "param_max_features": "max_features"}, inplace = True)
PCA_forest_grid_search_results["total_time"] = PCA_forest_grid_search_results["mean_fit_time"] + PCA_forest_grid_search_results["mean_score_time"]
PCA_forest_grid_search_results["standardized_time"] = PCA_forest_grid_search_results.total_time.map(lambda x: x / sum(PCA_forest_grid_search_results.total_time))
PCA_forest_grid_search_results.loc[:, ["rank_test_score", "max_depth", "max_features", "mean_test_score", "mean_train_score", "mean_fit_time", "mean_score_time", "total_time", "standardized_time"]].sort_values(by = "rank_test_score")

In [ ]:
# Plot the optimization of the correct PCA transformation in 3D

fig = plt.figure(figsize = (10, 10))
ax = fig.gca(projection='3d')

plot3d_title = "*Correct* PCA-transformed Random Forest cross-validation via GridSearch\n"
plot3d_title += "F1 score ≈ " + str(round(PCA_forest_score, 6))
plot3d_title += " at max_depth = " + str(PCA_forest_depth) + " and max_features = " + str(PCA_forest_features)

ax.set_title(plot3d_title)
ax.set_xlabel("max_depth", color = "r")
ax.set_ylabel("max_features", color = "b")
ax.set_zlabel("F1 score", color = "g")

ax.plot(PCA_forest_grid_search_results["max_depth"], PCA_forest_grid_search_results["max_features"],
           PCA_forest_grid_search_results["mean_test_score"], color = "blue", label = "Hyperparameter testing path")

ax.scatter(PCA_forest_depth, PCA_forest_features, PCA_forest_score, s = 256, marker = "o",
           color = "#ff0000cc", label = "Optimal depth, features")

depth_space =    np.linspace(PCA_forest_grid_search_results["max_depth"].min(),
                             PCA_forest_grid_search_results["max_depth"].max() + 1, 120)
features_space = np.linspace(PCA_forest_grid_search_results["max_features"].min(),
                             PCA_forest_grid_search_results["max_features"].max() + 1, 120)

plot_x, plot_y = np.meshgrid(depth_space, features_space)
score_plane = 0 * plot_x + 0 * plot_y + PCA_forest_score

ax.plot_wireframe(plot_x, plot_y, score_plane, rcount = 6, ccount = 6, color = "#00884466")

ax.legend(loc = "lower right")
ax.view_init(12, 45)
plt.savefig("PCA_hyperparameters.png")
plt.savefig("PCA_hyperparameters.pdf")

In [ ]:
# Plot the optimization of the basic Random Forest Classifier in 3D — Tracking time

fig = plt.figure(figsize = (10, 10))
ax = fig.gca(projection='3d')

ax.set_xlabel("\nProcessing time (standardized)", color = "r")
ax.set_ylabel("\nTrain minus test ~ Overfitting", color = "b")
ax.set_zlabel("\nF1 score", color = "g")

ax.plot(PCA_forest_grid_search_results["standardized_time"],
        PCA_forest_grid_search_results["mean_train_score"] - PCA_forest_grid_search_results["mean_test_score"],
        PCA_forest_grid_search_results["mean_test_score"], color = "blue", label = "Hyperparameter testing path")

winner_index = PCA_forest_grid_search_results["mean_test_score"].idxmax()

x_locate = PCA_forest_grid_search_results.standardized_time[winner_index]
y_locate = PCA_forest_grid_search_results.mean_train_score[winner_index] - PCA_forest_grid_search_results.mean_test_score[winner_index]

ax.scatter(x_locate, y_locate, PCA_forest_score, s = 256, marker = "o", color = "#ff0000cc",
           label = "Optimal depth, features")

depth_space =    np.linspace(PCA_forest_grid_search_results["standardized_time"].min(),
                             PCA_forest_grid_search_results["standardized_time"].max(), 120)
features_space = np.linspace(min(PCA_forest_grid_search_results["mean_train_score"] - PCA_forest_grid_search_results["mean_test_score"]),
                             max(PCA_forest_grid_search_results["mean_train_score"] - PCA_forest_grid_search_results["mean_test_score"]), 120)

plot3d_title = "*Correctly* reduced random forest cross-validation | "
plot3d_title += "F1 score ≈ " + str(round(PCA_forest_score, 6))
plot3d_title +=  "\nStandardized time ≈ " + str(round(x_locate, 6))
plot3d_title += " | Train minus test ≈ " + str(round(y_locate, 6))
ax.set_title(plot3d_title)

plot_x, plot_y = np.meshgrid(depth_space, features_space)
score_plane = 0 * plot_x + 0 * plot_y + PCA_forest_score

ax.plot_wireframe(plot_x, plot_y, score_plane, rcount = 8, ccount = 7, color = "#00884466")

ax.legend(loc = "lower right")
ax.view_init(12, 45)
plt.savefig("PCA_hyperparameters_time.png")
plt.savefig("PCA_hyperparameters_time.pdf")

In [ ]:
# PCA_forest = reduced_forest_fit(X_PCA_train, y_train, PCA_forest_features)
# print("Training set score:\t{:.6f}".format(PCA_forest.score(X_PCA_train, y_train)))
# print("Test set score:\t\t{:6f}".format(PCA_forest.score(X_PCA_test, y_test)))

PCA_forest = reduced_forest_fit(X_PCA_train, y_train, PCA_forest_depth, PCA_forest_features)

# Forgo these outputs in anticipation of the scorecard
# print("Training set score:\t{:.6f}".format(PCA_forest.score(X_PCA_train, y_train)))
# print("Test set score:\t\t{:6f}".format(PCA_forest.score(X_PCA_test, y_test)))

In [ ]:
crossval_scoring(PCA_forest, "*Correct* PCA transformation", "correct_PCA", X_PCA_train)

In [ ]:
scorecard(PCA_forest, X_PCA_train, X_PCA_test, "correct_PCA")

In [ ]:
purple_haze(y_test, PCA_forest.predict(X_PCA_test), classes = PCA_forest.classes_,
            title = "PCA random forest classifier", file_name = "PCA_forest")

In [ ]:
purple_haze(y_test, PCA_forest.predict(X_PCA_test), classes = PCA_forest.classes_, standardize = True,
            title = "PCA random forest classifier — Standardized", file_name = "PCA_forest_standardized")

In [ ]:
# Stop the clock on the correct PCA transformation

PCA_elapsed_time = time.monotonic() - PCA_start_time

*Bonus material*

Having run our random forest classifier and _two_ versions of its 95% transformation through PCA (one flawed and the other correct), we may now turn our attention to alternative classifiers. We start with a stochastic gradient descent classifier (SGD).

In [ ]:
# Set up a stochastic gradient descent (SGD) classifier
# Start the clock on SGD

SGD_start_time = time.monotonic()

# Binary classifier via sample code — Too simple for this exercise
# y_train_9 = (y_train == 9)  # True for 9's, false for every other
# y_test_9  = (y_test == 9)


sgd = SGDClassifier(random_state = RANDOM_SEED)
# sgd.fit(X_train, y_train)

In [ ]:
crossval_scoring(sgd, "Stochastic gradient descent", "sgd", X_train)

In [ ]:
scorecard(sgd, X_train, X_test, "sgd")

In [ ]:
purple_haze(y_test, sgd.predict(X_test), classes = sgd.classes_,
            title = "Stochastic gradient descent", file_name = "sgd")

In [ ]:
purple_haze(y_test, sgd.predict(X_test), classes = sgd.classes_, standardize = True, 
            title = "Stochastic gradient descent — Standardized", file_name = "sgd_standardized")

In [ ]:
# Stop the clock on Stochastic Gradient Descent

SGD_elapsed_time = time.monotonic() - SGD_start_time

We wish to complete our analysis by taking three "final steps." We will keep a running clock on these processes as well.

1: First, we wish to demonstrate a simple technique for ensuring that PCA analysis is properly conducted. Putting SciKit-Learn's Pipeline() to its original use, wholly apart from GridSearchCV(), allows us to _correctly_ reduce MNIST data through PCA _and_ to fit a PCA-ready version of our streamlined random forest model optimized for 154 features, all in a single step. We will use the cross-validation and scorecard routines developed earlier in this code and report its results in a final dataframe reporting all cross-validation results and and performance metrics.

2: Although the task of illustrating some difficult portions of our best performing random forest engine is not trivial, we think it's worth an extra few minutes to show a visual confusion matrix illustrating 4's and 9's that were correctly identified, as well as instances of each of these digits that were confused for the other.

3: Finally, we will produce a Final Pipeline comparing the performance and resource-consumption effects of PCA-dimension reduction as well as _scaling_ on random forest models. Scaling has almost no impact on random forest models. However, it has dramatic positive — and negative — effects on stochastic gradient descent. One simply should not generalize the impact of preprocessing on one class of machine-learning models (such as SGD and its linear cousins) onto another class (such as random forests).

In [ ]:
# Start the clock on final steps, including error analysis of 4-9 confusion
# And, of course, we want to lay a *** Final Pipeline ***

final_steps_start_time = time.monotonic()

In [ ]:
# Here we fit and score a simple Pipeline that combines *proper* PCA reduction with our PCA-optimized model

PCA_pipe = Pipeline([("reduction", PCA(n_components = 0.95)), ("model", PCA_forest)])
PCA_pipe.fit(X_train, y_train)
PCA_pipe.score(X_test, y_test)

In [ ]:
# Now we will score the PCA_pipe, directly on X_train instead of the a separately fit and transformed data set

crossval_scoring(PCA_pipe, "PCA Pipeline", "PCA_pipe", X_train)

In [ ]:
# The same process is repeated for the performance metric scorecard
# The final dataframes for the crossval and performance metric scorecards will reflect all results

scorecard(PCA_pipe, X_train, X_test, "PCA_pipe")

In [ ]:
# Error analysis — This is time counted against the "Final Steps" tally
# But we will profile it with the decorator function so that we know how much time it consumes

@profile
def error_analysis(cl_a = 4, cl_b = 9):

    # cl_a, cl_b = 4, 9
    y_train_pred = cross_val_predict(forest, X_train, y_train, cv=3)
    X_aa = X_train[(y_train == cl_a) & (y_train_pred == cl_a)]
    X_ab = X_train[(y_train == cl_a) & (y_train_pred == cl_b)]
    X_ba = X_train[(y_train == cl_b) & (y_train_pred == cl_a)]
    X_bb = X_train[(y_train == cl_b) & (y_train_pred == cl_b)]

    plt.figure(figsize=(8, 8))
    plt.subplot(221); plot_digits(X_aa[:25], images_per_row=5)
    plt.subplot(222); plot_digits(X_ab[:25], images_per_row=5)
    plt.subplot(223); plot_digits(X_ba[:25], images_per_row=5)
    plt.subplot(224); plot_digits(X_bb[:25], images_per_row=5)
    plt.savefig("error_analysis_digits_plot.png")
    plt.savefig("error_analysis_digits_plot.pdf")

# If time becomes an issue, we can comment out the function call
error_analysis(4, 9)

In [ ]:
@profile
def preprocessing_pipeline(features = X_train, target = y_train):
    
    forest_pipe = Pipeline([("reduction", PCA(n_components = 0.95)), ("scaling", StandardScaler()),
                            ("model", RandomForestClassifier())])

    param_grid = [{"model": [forest], "reduction": [None], "scaling": [None, StandardScaler(), Normalizer(), MinMaxScaler(), RobustScaler()]},
                  {"model": [PCA_forest], "reduction": [PCA(n_components = 0.95)], "scaling": [None, StandardScaler(), Normalizer(), MinMaxScaler(), RobustScaler()]},
                  {"model": [sgd], "reduction": [None], "scaling": [None, StandardScaler(), Normalizer(), MinMaxScaler(), RobustScaler()]}
                 ]

    grid_search_forest_pipe = GridSearchCV(forest_pipe, param_grid, cv = 3, return_train_score = True,
                                           scoring = f1_scorer, iid = False)

    grid_search_forest_pipe.fit(features, target)
    pipe_params  = grid_search_forest_pipe.best_params_
    pipe_score   = grid_search_forest_pipe.best_score_
    pipe_results = grid_search_forest_pipe.cv_results_
    
    return pipe_params, pipe_score, pipe_results

pipe_params, pipe_score, pipe_results = preprocessing_pipeline(X_train, y_train)

print("Results of the standard scaling Pipeline:\n")
print("Best parameter:\t{}".format(pipe_params))
print("Best cv score:\t{:.6f}".format(pipe_score))

In [ ]:
# pipe_results = grid_search_forest_pipe.cv_results_
pipe_frame = pd.DataFrame(pipe_results)
pipe_frame

In [ ]:
pipe_frame["param_model"] = pipe_frame["param_model"].map(lambda x: str(x))
for record_no in range(len(pipe_frame.param_model)):
    model_name = pipe_frame.loc[record_no, "param_model"]
    if model_name.partition("(")[0] == "SGDClassifier":
        pipe_frame.loc[record_no, "param_model"] = "SGD"
    elif str.find(model_name, str(forest_features)) > 0:  # A regular random forest model
        pipe_frame.loc[record_no, "param_model"] = "RandomForest"
    else:
        pipe_frame.loc[record_no, "param_model"] = "PCA"

In [ ]:
pipe_frame.rename(columns = {"param_model": "model", "param_reduction": "reduction", "param_scaling": "scaling",
                             "mean_test_score": "test_score", "mean_train_score": "train_score",
                             "mean_fit_time": "fit_time", "mean_score_time": "score_time"}, inplace = True)
pipe_frame["scaling"] = pipe_frame["scaling"].map(lambda x: str(x).partition("(")[0])
pipe_frame["train_minus_test"] = pipe_frame["train_score"] - pipe_frame["test_score"]
pipe_frame["total_time"] = pipe_frame["fit_time"] + pipe_frame["score_time"]
pipe_frame["standardized_time"] = pipe_frame["total_time"].map(lambda x: x / pipe_frame.total_time.sum())

In [ ]:
final_pipe = pipe_frame.loc[:, ["rank_test_score", "model", "scaling", "test_score", "train_score", "train_minus_test", "fit_time", "score_time", "total_time", "standardized_time"]].sort_values(by = "rank_test_score")
final_pipe

In [ ]:
# Plot the Pipeline() in 3D — Test score as a function of processing time and overfitting

fig = plt.figure(figsize = (12, 12))
ax = fig.gca(projection='3d')

ax.set_xlabel("\nProcessing time (standardized)", color = "r")
ax.set_ylabel("\nTrain minus test ~ Overfitting", color = "b")
ax.set_zlabel("\nF1 score", color = "g")

winner_index = final_pipe["test_score"].idxmax()
winning_model = final_pipe.loc[winner_index, "model"]
if final_pipe.loc[winner_index, "scaling"] == None:
    winning_scaling = "None"
else:
    winning_scaling = final_pipe.loc[winner_index, "scaling"]

winning_label = "Best model: " + winning_model + " | Scaling: " + winning_scaling

def plot_model_lines(model_name, model_color):
    ax.plot(final_pipe[final_pipe.model == model_name]["standardized_time"],
        final_pipe[final_pipe.model == model_name]["train_minus_test"],
        final_pipe[final_pipe.model == model_name]["test_score"],
        color = model_color, label = model_name)

def scatter_model_dots(model_name, model_color, model_marker):
    ax.scatter(final_pipe[final_pipe.model == model_name]["standardized_time"],
               final_pipe[final_pipe.model == model_name]["train_minus_test"],
               final_pipe[final_pipe.model == model_name]["test_score"],
               s = 96, marker = model_marker, color = model_color, label = model_name)
    
scatter_model_dots("RandomForest", "red", "o")
scatter_model_dots("PCA", "blue", "^")
plot_model_lines("SGD", "brown")

x_locate = final_pipe.standardized_time[winner_index]
y_locate = final_pipe.train_minus_test[winner_index]

# ax.scatter(x_locate, y_locate, pipe_score, s = 256, marker = "o", color = "#ff0000cc",
#            label = winning_label)

time_space =    np.linspace(final_pipe["standardized_time"].min(), final_pipe["standardized_time"].max(), 120)
overfit_space = np.linspace(final_pipe["train_minus_test"].min(), final_pipe["train_minus_test"].max(), 120)

plot3d_title = winning_label + " | F1 score ≈ " + str(round(pipe_score, 6))
plot3d_title +=  "\nStandardized time ≈ " + str(round(x_locate, 6))
plot3d_title += " | Train minus test ≈ " + str(round(y_locate, 6))
ax.set_title(plot3d_title)

plot_x, plot_y = np.meshgrid(time_space, overfit_space)
score_plane = 0 * plot_x + 0 * plot_y + pipe_score

ax.plot_wireframe(plot_x, plot_y, score_plane, rcount = 8, ccount = 7, color = "#00884466")

# ax.plot_surface(plot_x, plot_y, score_plane, color = "#ffff0066")

ax.legend(loc = "lower right")
ax.view_init(12, 45)
plt.savefig("final_pipe.png")
plt.savefig("final_pipe.pdf")

In [ ]:
# We're almost done! Time to harvest all of that pandas work and report their results.
# First, the dataframe showing cross-valuation results …

crossval_frame

In [ ]:
# Now, the dataframe consolidating results from scorecards on each of our models

scorecard_frame

In [ ]:
# Stop the clock. This is it.

final_steps_elapsed_time = time.monotonic() - final_steps_start_time

In [ ]:
print("{:>60}\n".format("*** FINAL TIMEKEEPING SCORECARD ***"))

print("{:>36}\t{:12.6f}".format("Random forest time", RF_elapsed_time))
print("{:>36}\t{:12.6f}".format("(Flawed) PCA-transformation time", reduced_elapsed_time))
print("{:>36}\t{:12.6f}".format("*Correct* PCA-transformation time", PCA_elapsed_time))
print("{:>36}\t{:12.6f}".format("Stochastic gradient descent time", SGD_elapsed_time))
print("{:>36}\t{:12.6f}".format("Error analysis/Final Pipeline time", final_steps_elapsed_time))
print("{:>36}\t{:12.6f}\n".format("TOTAL", RF_elapsed_time + reduced_elapsed_time + PCA_elapsed_time + SGD_elapsed_time + final_steps_elapsed_time))
print_prof_data()

In [ ]:
# We can also export PROF_DATA to a DataFrame for nicer display and possibly also for future use
# Display the refined dataframe so that …
#   1. It suppresses the individual times, without deleting that information
#   2. It is sorted in descending order of total time taken

profile_frame = pd.DataFrame(PROF_DATA)
profile_frame = profile_frame.transpose()
profile_frame.columns = ["calls", "times"]
profile_frame["total"] = profile_frame.times.map(sum)
profile_frame["average"] = profile_frame.times.map(np.mean)
profile_frame["std_dev"] = profile_frame.times.map(np.std)
profile_frame["maximum"] = profile_frame.times.map(max)
profile_frame["minimum"] = profile_frame.times.map(min)

print("\t*** Timekeeping summary of profiled functions ***\n")
print("{:d} profiled functions took {:d} calls for {:.6f} total seconds, an average of {:.6f}.".format(len(profile_frame), profile_frame.calls.sum(), profile_frame.total.sum(), profile_frame.total.mean()))

profile_frame.loc[:, ["calls", "total", "average", "std_dev", "maximum", "minimum"]].sort_values(by = "total", ascending = False)